In [42]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

import sqlite3
import pandas as pd
from budget_functions import setup

In [24]:
if os.path.exists('secrets/creds_v2.json'):
    print('si')

folder_name = 'ClevMoney'

si


In [22]:
def get_latest_clevmoney_file(placeholder=None):
    folder_name = 'ClevMoney'
    SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "secrets/creds_v2.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)

        # Search for the folder by name
    folder_query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder'"
    folder_results = service.files().list(q=folder_query, fields="files(id)").execute()
    folder_id = folder_results['files'][0]['id'] if folder_results['files'] else None

    if not folder_id:
        raise ValueError(f"Folder '{folder_name}' not found")

    # Search for the most recent file in the folder
    file_query = f"'{folder_id}' in parents and trashed=false"
    file_results = service.files().list(q=file_query, orderBy='createdTime desc', pageSize=1, fields="files(id, name)").execute()
    file_id = file_results['files'][0]['id'] if file_results['files'] else None

    if not file_id:
        raise ValueError(f"No files found in folder '{folder_name}'")

    # Retrieve the file content
    request = service.files().get_media(fileId=file_id)
    file_content = io.BytesIO()
    downloader = MediaIoBaseDownload(file_content, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()

    # Write the content to a local file
    with open("downloaded_file", "wb") as f:
        f.write(file_content.getvalue())

    print("File downloaded successfully.")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=887686446977-mtu94vcvo8iqu9o74oegf2v3ti4k2fnf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52475%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=8Ifa8P4Fa2f2woHICCsacOuIr7cnh3&access_type=offline


In [45]:
setup('downloaded_file')

,s_date,s_time,s_where,s_cate,s_subcate,s_price,Month,Year
0,2022-05-02,15:13:00,Tesco,47.0,56.0,9.70,5,2022
1,2022-05-02,15:14:00,Santander,21.0,49.0,90.00,5,2022
2,2022-05-03,15:37:00,Tesco,1.0,50.0,3.20,5,2022
3,2022-05-03,22:19:00,RyanAir (Amsterdam),51.0,54.0,90.82,5,2022
4,2022-05-03,22:19:00,EasyJet (Poland),51.0,54.0,126.66,5,2022
...,...,...,...,...,...,...,...,...
2280,2024-06-04,12:44:00,Nisa,47.0,58.0,4.31,6,2024
2281,2024-06-04,20:06:00,M&S,47.0,60.0,11.07,6,2024
2282,2024-06-06,00:00:00,Thames Water,119.0,124.0,13.00,6,2024
2283,2024-06-06,00:00:00,Octopus Energy,119.0,120.0,60.45,6,2024


In [35]:
def query_sqlite_from_bytes(file_content, query):
    """
    Load a SQLite database from a byte string and execute a query.

    Parameters:
    - file_content: The content of the SQLite database as bytes.
    - query: The SQL query to execute.

    Returns:
    - result: The result of the query as a pandas DataFrame.
    """
    # Create an in-memory SQLite database and load the file content
    conn = sqlite3.connect(':memory:')

    try:
        # Load the database content from bytes into the in-memory database
        with io.BytesIO(file_content) as f:
            for line in f:
                conn.executescript(line.decode())

        # Execute the query
        result = pd.read_sql_query(query, conn)
    except Exception as e:
        print(f"An error occurred: {e}")
        result = None
    finally:
        conn.close()

    return result


In [39]:
# Query the SQLite database from the file content
query = "SELECT 8 FROM sqlite_master WHERE type='table'"
result_df = query_sqlite_from_bytes(file_content.read(), query)

# Display the result
print(result_df)

Empty DataFrame
Columns: [name]
Index: []
